<a href="https://colab.research.google.com/github/ahalamora1981/chatgpt_demo01/blob/main/online_demo_battle_qwen_chatglm2_baichuan_via_modelscop_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio_client

In [ ]:
from gradio_client import Client
import concurrent.futures

In [ ]:
qwen_client = Client("https://modelscope.cn/api/v1/studio/qwen/Qwen-7B-Chat-Demo/gradio/")
chatglm2_client = Client("https://modelscope.cn/api/v1/studio/AI-ModelScope/ChatGLM6B-unofficial/gradio/")
baichuan_client = Client("https://modelscope.cn/api/v1/studio/baichuan-inc/Baichuan-13B-Chatdemo/gradio/")

Loaded as API: https://modelscope.cn/api/v1/studio/qwen/Qwen-7B-Chat-Demo/gradio/ ✔
Loaded as API: https://modelscope.cn/api/v1/studio/AI-ModelScope/ChatGLM6B-unofficial/gradio/ ✔
Loaded as API: https://modelscope.cn/api/v1/studio/baichuan-inc/Baichuan-13B-Chatdemo/gradio/ ✔


In [ ]:
# 去除非UTF-8的字符
def remove_non_utf8_chars(input_str):
    # 先将字符串编码为字节对象，忽略无效字符
    encoded_str = input_str.encode('utf-8', 'ignore')
    # 再将字节对象解码回字符串
    return encoded_str.decode('utf-8')

# 验证去除非UTF-8字符的效果

In [ ]:
json_path = result[1]

In [ ]:
with open (json_path, 'r') as f:
    data = f.read()
data

'[["\\u4f60\\u597d", "\\u4f60\\u597d\\ud83d\\udc4b\\uff01\\u6211\\u662f\\u4eba\\u5de5\\u667a\\u80fd\\u52a9\\u624b ChatGLM2-6B\\uff0c\\u5f88\\u9ad8\\u5174\\u89c1\\u5230\\u4f60\\uff0c\\u6b22\\u8fce\\u95ee\\u6211\\u4efb\\u4f55\\u95ee\\u9898\\u3002"]]'

In [ ]:
data = eval(data)
data

[['你好', '你好\ud83d\udc4b！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。']]

In [ ]:
for i in range(len(data)):
    data[i][1] = remove_non_utf8_chars(data[i][1])
data

[['你好', '你好！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。']]

# Chat函数

In [ ]:
def chat(query, client, json_path):
    try:
        result = client.predict(
            query, # str  in 'Input' Textbox component
            json_path, # str (filepath to JSON file) in Chatbot component
            fn_index=0
        )

        json_path = result[1]

        with open (json_path, 'r') as f:
            data = f.read()

        data = eval(data)

        for i in range(len(data)):
            data[i][1] = remove_non_utf8_chars(data[i][1])

    except Exception as e:
        if json_path:
            with open (json_path, 'r') as f:
                data = f.read()
        else:
            data = []

        data.append([query, e])

    return data, json_path

# 常规调用方式（串行）

In [ ]:
json_qwen = ''
json_chatglm2 = ''
json_baichuan = ''

In [ ]:
query = "太阳系有哪些行星？其中哪些是气态的？"

In [ ]:
%%time

data_qwen, json_qwen = chat(query, qwen_client, json_qwen)
data_chatglm2, json_chatglm2 = chat(query, chatglm2_client, json_chatglm2)
data_baichuan, json_baichuan = chat(query, baichuan_client, json_baichuan)

for i in range(len(data_chatglm2)):
    human = data_chatglm2[i][0]
    qwen = data_qwen[i][1]
    chatglm2 = data_chatglm2[i][1]
    baichuan = data_baichuan[i][1]

    print(f'\nHuman: {human}\n\n' \
          + f'Qwen: {qwen}\n\n' \
          + f'ChatGLM2: {chatglm2}\n\n' \
          + f'Baichuan: {baichuan}\n\n' \
          + '**************************************************\n')


Human: 太阳系有哪些行星？其中哪些是气态的？

Qwen: 太阳

ChatGLM2: 太阳系共有8颗行星，它们按照距离太阳的远近依次是：

1. 水星（Mercury）：离太阳最近的行星，同时也是太阳系中最小的行星。它的表面温差很大，夜晚极度寒冷，白天则极度炎热。

2. 金星（Venus）：金星是太阳系中第二颗行星，也是离地球最近的行星。金星的大气层厚度较大，由80%的二氧化碳和10%的云层中的硫酸组成，形成了严重的温室效应，使得金星的表面温度极高。

3. 地球（Earth）：地球是我们所生活的星球，是太阳系中唯一已知存在生命的星球。地球有着丰富的水资源，气候适宜，自然环境多样。

4. 火星（Mars）：火星是太阳系中重要的行星，它的表面特征与地球相似，但有所差异。火星上曾有液态水存在的迹象，因此科学家认为火星可能曾经有过适宜生命存在的条件。

5. 木星（Jupiter）：木星是太阳系中最大的行星，它主要由氢和氦组成，表面温度较低。木星以其巨大而美丽的风暴而闻名，同时也是太阳系中重要的气体成分。

6. 土星（Saturn）：土星是太阳系中第二大行星，也是最轻的行星。土星以其巨大而美丽的环系统著名，这些环主要由冰块和岩石组成。

7. 天王星（Uranus）：天王星是太阳系中最小的行星，以其倾斜的轴线和奇特的形状而闻名。天王星的大气层主要由氢和氦组成，还有少量的甲烷和二氧化碳。

8. 海王星（Neptune）：海王星是太阳系中最远的行星。它的表面主要由冰和冰块组成，还有少量的石质物质。海王星的大气层主要由氢和氦组成，还有少量的甲烷和二氧化碳。

Baichuan: 太阳系共有8个行星，它们按离太阳的距离从近到远依次是：水星、金星、地球、火星、木星、土星、天王星和海王星。在这8个行星中，有4个是气态巨星，它们分别是：木星、土星、天王星和海王星。这些气态巨星主要由气体(如氢、氦)和液体组成，它们的质量和体积都非常庞大。

**************************************************

CPU times: user 1.74 s, sys: 267 ms, total: 2.01 s
Wall time: 1min 21s


# Concurrent 调用方式（并行）

In [ ]:
json_qwen = None
json_chatglm2 = None
json_baichuan = None

In [ ]:
query = "太阳系有哪些行星？其中哪些是气态的？"

In [ ]:
%%time

# Run the chat calls concurrently
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_qwen = executor.submit(chat, query, qwen_client, json_qwen)
    future_chatglm2 = executor.submit(chat, query, chatglm2_client, json_chatglm2)
    future_baichuan = executor.submit(chat, query, baichuan_client, json_baichuan)

# Get the results from the futures
data_qwen, json_qwen = future_qwen.result()
data_chatglm2, json_chatglm2 = future_chatglm2.result()
data_baichuan, json_baichuan = future_baichuan.result()

for i in range(len(data_qwen)):
    human = data_qwen[i][0]
    qwen = data_qwen[i][1]
    chatglm2 = data_chatglm2[i][1]
    baichuan = data_baichuan[i][1]

    print(f'\nHuman: {human}\n\n' \
          + f'【Qwen】: {qwen}\n\n' \
          + f'【ChatGLM2】: {chatglm2}\n\n' \
          + f'【Baichuan】: {baichuan}\n\n' \
          + '**************************************************\n')


Human: 太阳系有哪些行星？其中哪些是气态的？

【Qwen】: 太阳

【ChatGLM2】: 太阳系共有8颗行星,它们按照距离太阳的远近依次是:

1. 水星(Mercury)
2. 金星(Venus)
3. 地球(Earth)
4. 火星(Mars)
5. 木星(Jupiter)
6. 土星(Saturn)
7. 天王星(Uranus)
8. 海王星(Neptune)

其中,水星、金星、地球和火星是气态行星,另外,木星、土星、天王星和海王星是巨行星。

【Baichuan】: 太阳系有8个行星，它们按离太阳的距离从近到远依次是：水星、金星、地球、火星、木星、土星、天王星和海王星。在这8个行星中，只有木星和土星被认为是气态巨星。

1. 木星(Jupiter)：木星是太阳系中最大的行星，主要由氢和氦组成。它的内部结构类似于一个巨大的气体球体，中心有一个液态金属核心。木星的大气层非常厚，主要成分是氢气和氦气，还含有少量的甲烷、水和其他化合物。

2. 土星(Saturn)：土星是太阳系中第二大的行星，主要成分同样是氢和氦。与木星相似，土星也有一层厚厚的气体外壳，但它的外形更接近一个圆球。土星的大气层主要由氢气和氦气组成，还包含一定比例的甲烷和氨。

这两颗气态巨星的特点是它们的质量主要集中在大气层，而地幔和地核相对较小。它们的引力较弱，因此卫星数量较多且密度较低。此外，它们的磁场强度也很大，这有助于解释它们强烈的极光现象。

**************************************************

CPU times: user 714 ms, sys: 117 ms, total: 830 ms
Wall time: 29 s
